In [1]:
import json
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# Conectar ao banco covid_etl


# Carregar configurações de conexão
with open('../config.json', 'r') as config_file:
    config = json.load(config_file)

# Criar engine SQLAlchemy
engine = create_engine(
    f'postgresql+psycopg2://{config["user"]}:{config["password"]}@{config["host"]}:{config["port"]}/covid_etl'
)


FileNotFoundError: [Errno 2] No such file or directory: '../config.json'

In [ ]:
# criando função que fará as consultas e retorna o dataframe
def get_dataframe(query):
    df = pd.read_sql(query, engine)
    return df


In [ ]:
# CONSULTANDO QUANTAS PESSOAS TOMARAM DOSE INFANTIL (PEDIÁTRICA) RELACIONANDO COM VACINA_LOTE
query = '''
SELECT
    "idade (anos)", grupo, data_vacinacao, fabricante
FROM analytics.pessoas_vacinadas pv
JOIN analytics.vacina_lote vl ON pv.lote = vl.lote
WHERE vl.pediatrica = TRUE and pv."idade (anos)" > 13;
'''
get_dataframe(query).head()

In [ ]:
# CONSULTANDO QUANTAS PESSOAS TOMARAM DOSE INFANTIL (PEDIÁTRICA) RELACIONANDO COM VACINA_LOTE
query = '''
SELECT
  COUNT(*) AS total_vacinados_pediatrica,
  SUM(CASE WHEN pv."idade (anos)" <= 13 THEN 1 ELSE 0 END) AS total_criancas,
  SUM(CASE WHEN pv."idade (anos)" > 13 THEN 1 ELSE 0 END) AS total_nao_criancas,
  ROUND(
    100.0 * SUM(CASE WHEN pv."idade (anos)" <= 13 THEN 1 ELSE 0 END) / COUNT(*),
    2
  ) AS percentual_criancas
FROM analytics.pessoas_vacinadas pv
JOIN analytics.vacina_lote vl ON pv.lote = vl.lote
WHERE vl.pediatrica = TRUE;
'''
get_dataframe(query).head()

Insight: Doses pediátricas foram registradas como aplicadas em adultos, indicando possível erro de registro ou classificação de lote.

-  Mais de 70% das doses pediátricas foram aplicadas em pessoas fora da faixa etária infantil

- Dos 42 registros de vacinas pediátricas, apenas 12 foram aplicados em crianças (≤13 anos), o que representa 28,57% do total. Os outros 30 registros (71,43%) correspondem a pessoas com mais de 13 anos, o que indica uma inconsistência relevante no uso ou registro dos lotes pediátricos.

- A análise identificou que doses classificadas como pediátricas (pediatrica = TRUE) foram aplicadas em pessoas com mais de 13 anos, incluindo adultos com idades entre 28 e 60 anos. Isso é incompatível com a indicação da vacina pediátrica, que é destinada a crianças, normalmente até 11 ou 12 anos.

In [ ]:
# CONSULTANDO PERCENTUAL DE VACINADOS POR SEXO
query = '''
SELECT
    sexo,
    COUNT(*) AS total,
    ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER (), 2) AS percentual
FROM analytics.pessoas_vacinadas
GROUP BY sexo
'''
get_dataframe(query).head()

Insight: A distribuição por sexo entre os vacinados está dentro do esperado, com leve predominância feminina.
- Mulheres representam 50,85% dos vacinados, enquanto homens representam 49,03%, o que configura uma distribuição equilibrada e dentro da normalidade. Esse leve predomínio feminino é comum em campanhas de vacinação, refletindo tanto a maior presença de mulheres em setores essenciais quanto uma tendência histórica de maior adesão à saúde preventiva.

- O percentual de registros em branco ou com classificação "OUTROS" é muito baixo (apenas 0,12%), o que indica boa qualidade na coleta dos dados de sexo.

In [ ]:
# CONSULTANDO DISTRIBUIÇÃO DE VACINADOS POR FAIXA ETARIA

query = '''
SELECT
    CASE
        WHEN "idade (anos)" BETWEEN 0 AND 17 THEN 'Menor de 18'
        WHEN "idade (anos)" BETWEEN 18 AND 29 THEN '18-29'
        WHEN "idade (anos)" BETWEEN 30 AND 39 THEN '30-39'
        WHEN "idade (anos)" BETWEEN 40 AND 49 THEN '40-49'
        WHEN "idade (anos)" BETWEEN 50 AND 59 THEN '50-59'
        WHEN "idade (anos)" BETWEEN 60 AND 69 THEN '60-69'
        ELSE '70+'
    END AS faixa_etaria,
    COUNT(*) AS total_vacinados
FROM analytics.pessoas_vacinadas
GROUP BY faixa_etaria
ORDER BY total_vacinados DESC
'''
get_dataframe(query).head()

Insight: A faixa 40–49 anos representa o maior número absoluto de vacinados (29.316), superando até a faixa de 70+ (25.681), que seria normalmente priorizada por estar em maior risco.

Possíveis interpretações:
- Demografia local: Pode haver mais pessoas entre 40–49 anos na população da sua área do que acima de 70 anos — o que faria sentido em muitos contextos populacionais, especialmente em países mais jovens como o Brasil.

- Maior adesão à vacinação entre adultos ativos: Indivíduos de 40–49 anos podem ter maior consciência da importância da vacinação, acesso facilitado (por trabalharem ou terem mais mobilidade), ou exigência de vacinação para trabalho.

- Barreiras para os idosos: A faixa de 70+ pode ter enfrentado obstáculos logísticos, como dificuldades de locomoção, problemas de agendamento digital ou hesitação vacinal.

In [ ]:
# CONSULTANDO O TOTAL DE DOSES APLICADAS POR FABRICANTE DE VACINA
query = '''
SELECT
    vl.fabricante,
    COUNT(*) AS total_doses,
    ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER (), 2) AS percentual
FROM
    analytics.pessoas_vacinadas pv
JOIN
    analytics.vacina_lote vl ON pv.lote = vl.lote
GROUP BY
    vl.fabricante
ORDER BY
    total_doses DESC;
'''
get_dataframe(query).head()

Insight: Predominância de um Fabricante e a Estratégia de Vacinação
A análise dos dados por fabricante revela qual imunizante foi o pilar da campanha de vacinação nesta base de dados. O resultado desta consulta permite observar:

- Concentração de Fornecedor: É possível que um único fabricante (como PFIZER, ASTRAZENECA, etc.) seja responsável por uma parcela majoritária das doses aplicadas. Isso pode indicar uma dependência de um fornecedor específico, seja por contratos de maior volume, disponibilidade logística ou por ter sido a primeira vacina a chegar em grande quantidade.

- Diversificação da Campanha: Se os percentuais forem bem distribuídos entre vários fabricantes, isso sugere uma estratégia de diversificação na aquisição de vacinas, o que pode mitigar riscos de falta de suprimento e atender a diferentes públicos ou recomendações de saúde.

- Validação de Dados: Esta visão também serve como uma validação cruzada. Por exemplo, se um fabricante que só produz vacinas para adultos aparecer em registros de crianças (ou vice-versa), isso reforçaria a suspeita de erros no registro dos lotes, como já foi observado na primeira análise sobre as doses pediátricas.

Em suma, este insight mapeia a "espinha dorsal" da campanha de vacinação, mostrando quais vacinas foram mais acessíveis e utilizadas pela população registrada.